In [1]:
import os.path

import pandas as pd
from src.all_in_one import *
threshold = "2022-12-31"
upper_threshold = "2024-01-01"

# UNIQUE SCREENING BENEFICIARIES RECORDS

In [2]:
# scr_dataset = Dataset(config_file, scr_sheet)
# scr = pd.read_csv(scr_path)
# unique_scr = scr.drop_duplicates(subset='rid')

# UNIQUE INTAKE BENEFICIARIES RECORDS

In [3]:
# int_dataset = Dataset(config_file, int_sheet)
# intake = pd.read_csv(int_path)
# dtype_trans(intake, int_dataset)
# intake.insert(4, "service", None)
# intake.sort_values(by=['nt_s1', 'nt_s2', 'nt_s3', 'nt_re'], ascending=True)
# unique_intake = intake.drop_duplicates(subset='rid', keep='last')

In [4]:
# intake[intake['fcid'] == 8110020103]

## PSYCHOSOCIAL INTAKE ASSESSMENT PROCESSING

In [5]:
# intake_23 = filter_df_by_date(intake, ["nt_s2"], threshold=threshold)
# intake_23 = intake_23[["rid", "fcid", "service","nt_s2", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake_23["service"] = "int"
# intake_23.rename(columns={"nt_s2": "date"}, inplace=True)
# intake_23

In [6]:
# intake3_23 = filter_df_by_date(intake, ["nt_s3"], threshold=threshold)
# intake3_23 = intake3_23[["rid", "fcid", "service","nt_s3", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake3_23["service"] = "int"
# intake3_23.rename(columns={"nt_s3": "date"}, inplace=True)

## PSYCHOSOCIAL RE-INTAKE ASSESSMENT PROCESSING

In [7]:
# reintake_23 = filter_df_by_date(intake, ["nt_re"], threshold=threshold)
# reintake_23 = reintake_23[["rid", "fcid", "service","nt_re", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# reintake_23["service"] = "re"
# reintake_23.rename(columns={"nt_re": "date"}, inplace=True)

In [8]:
# int23 = pd.concat([intake_23, intake3_23,reintake_23])

In [9]:
# int23 = pd.merge(int23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [10]:
# _intake23 = intake23[intake23['service'] == 'int']
# reintake23 = intake23[intake23['service'] == 're']
# _intake23_path= os.path.join(root, data, processed, "unvfvt23", "intBeneficiaries23.csv")
# reintake23_path= os.path.join(root, data, processed, "unvfvt23", "reintBeneficiaries23.csv")
# _intake23.to_csv(_intake23_path, index=False)
# reintake23.to_csv(reintake23_path, index=False)

## GROUP PSYCHOSOCIAL THERAPY COUNSELING PROCESSING

In [11]:
# gc_dataset = Dataset(config_file, gc_sheet)
# gc = pd.read_csv(gc_path)
# dtype_trans(gc, gc_dataset)

In [12]:
# gc23 = filter_df_by_date(gc, ['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10'], threshold)
# gc23 = gc23[gc23["gc_total"] >= 7]
# gc23_latestdate = gc23[['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10']].max(axis=1)
# gc23.insert(4, "service", "gc")
# gc23.insert(5, "date", gc23_latestdate)
# gc23

In [13]:
# gc23 = gc23[['rid', 'fcid', 'service', 'date', 'gc_total']]
# gc23 = pd.merge(gc23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# gc23 = pd.merge(gc23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# gc23

In [14]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gc23")
# gc23 = pd.read_csv(gc23_path)
# gc23

## INDIVIDUAL PSYCHOSOCIAL THERAPY COUNSELING PROCESSING

In [15]:
# ic_dataset = Dataset(config_file, ic_sheet)
# ic = pd.read_csv(ic_path)
# dtype_trans(ic, ic_dataset)
# ic

In [16]:
# ic23 = filter_df_by_date(ic, ic_dataset.dvars, threshold)
# ic23_latestdate = ic23[ic_dataset.dvars].max(axis=1)
# ic23.insert(4, "service", "ic")
# ic23.insert(5, "date", ic23_latestdate)
# ic23 = ic23[['rid', 'fcid', 'service', 'date', 'ic_total']]
# ic23 = pd.merge(ic23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ic23 = pd.merge(ic23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ic23

In [17]:
# int23_path = os.path.join(root, data, processed, "unvfvt23", "intBeneficiaries23.csv")
# int23.to_csv(path, index=False)
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# gc23.to_csv(gc23_path, index=False)

In [18]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# pd.read_csv(gc23_path)

## PSYCHOSOCIAL FOLLOW-UP ASSESSMENT PROCESSING

In [19]:
# psfua_dataset = Dataset(config_file, ptfua_sheet)
# psfua = pd.read_csv(psfua_path)
# psfua = dtype_trans(psfua, psfua_dataset)

In [20]:
# psfua323 = filter_df_by_date(psfua, ['fu_3m'], threshold)
# psfua323.insert(4, "service", "psfua3")
# psfua623 = filter_df_by_date(psfua, ['fu_6m'], threshold)
# psfua623 = filter_df_by_date(psfua623, ['fu_6m'], upper_threshold, comparison_type='b')
# psfua623.insert(4, "service", "psfua6")
# psfua1223 = filter_df_by_date(psfua, ['fu_12m'], threshold)
# psfua1223.insert(4, "service", "psfua12")

In [21]:
# psfua323 = psfua323[['rid', 'fcid', 'service', 'fu_3m']].copy()
# psfua323.rename(columns={'fu_3m': 'date'}, inplace=True)
# psfua623 = psfua623[['rid', 'fcid', 'service', 'fu_6m']].copy()
# psfua623.rename(columns={'fu_6m': 'date'}, inplace=True)
# psfua1223 = psfua1223[['rid', 'fcid', 'service', 'fu_12m']].copy()
# psfua1223.rename(columns={'fu_12m': 'date'}, inplace=True)
# psfua23 = pd.concat([psfua323, psfua623, psfua1223])

In [22]:
# psfua23 = pd.merge(psfua23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# psfua23 = pd.merge(psfua23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [23]:
# psfua23_path = os.path.join(root, data, processed, "unvfvt23", "psfuaBeneficiaries23.csv")
# psfua23.to_csv(psfua23_path, index=False)
# psfua23

## THERAPUTIC DOCUMENTATION PROCESSING

In [24]:
# td_dataset = Dataset(config_file, td_sheet)
# td = pd.read_csv(td_path)
# td = dtype_trans(td, td_dataset)

In [25]:
# td23 = filter_df_by_date(td, td_dataset.dvars, threshold)
# td23 = filter_df_by_date(td23, td_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# td23_latestdate = td23[td_dataset.dvars].max(axis=1)
# td23.insert(4, "service", "td")
# td23.insert(5, "date", td23_latestdate)
# td23 = td23[['rid', 'fcid', 'service', 'date']]
# td23 = pd.merge(td23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# td23 = pd.merge(td23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [26]:
# td23_path = os.path.join(root, data, processed, "unvfvt23", "tdBeneficiaries23.csv")
# td23.to_csv(td23_path, index=False)

## POST-EARTHQUAKE INTERVENTION PROCESSING

In [27]:
# pei_dataset = Dataset(config_file, pei_sheet)
# pei = pd.read_csv(pei_path)
# pei = dtype_trans(pei, pei_dataset)

In [28]:
# pei23 = filter_df_by_date(pei, pei_dataset.dvars, threshold)
# pei23 = filter_df_by_date(pei23, pei_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# pei23_latestdate = pei23[pei_dataset.dvars].max(axis=1)
# pei23.insert(4, "service", "pei")
# pei23.insert(5, "date", pei23_latestdate)
# pei23 = pei23[['rid', 'fcid', 'service', 'date', "pei_total"]]
# pei23 = pd.merge(pei23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# pei23 = pd.merge(pei23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [29]:
# pei23_path = os.path.join(root, data, processed, "unvfvt23", "peiBeneficiaries23.xlsx")
# pei23.to_csv(pei23_path, index=False)
# pei23.to_excel(pei23_path, index=False)

## GROUP PHYSIOTHERAPY PROCESSING

In [30]:
# gpt_dataset = Dataset(config_file, gpt_sheet)
# gpt = pd.read_csv(gpt_path)
# gpt = dtype_trans(gpt, gpt_dataset)
# non_nat_counts = gpt[gpt_dataset.dvars].notna().sum(axis=1)
# gpt['pt_g_total'] = np.where(gpt['pt_g_total'] == 0, non_nat_counts, gpt['pt_g_total'])

In [31]:
# gpt23 = filter_df_by_date(gpt, gpt_dataset.dvars, threshold)
# gpt23 = filter_df_by_date(gpt23, gpt_dataset.dvars, upper_threshold, comparison_type='b')
# gpt_mask = (gpt23[gpt_dataset.dvars] > threshold) & (gpt23[gpt_dataset.dvars] < upper_threshold)
# gpt23_latestdate = gpt23[gpt_dataset.dvars].where(gpt_mask).max(axis=1)
# gpt23.insert(4, "service", "gpt")
# gpt23.insert(5, "date", gpt23_latestdate)
# gpt23 = gpt23[['rid', 'fcid', 'service', 'date', 'pt_g_total']]

In [32]:
# gpt23 = pd.merge(gpt23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# gpt23 = pd.merge(gpt23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [33]:
# gpt23_path = os.path.join(root, data, processed, "unvfvt23", "gptBeneficiaries23.csv")
# gpt23.to_csv(gpt23_path, index=False)

## INDIVIDUAL PHYSIOTHERAPY PROCESSING

In [34]:
# ipt_dataset = Dataset(config_file, ipt_sheet)
# ipt = pd.read_csv(ipt_path)
# ipt = dtype_trans(ipt, ipt_dataset)
# non_nat_counts = ipt[ipt_dataset.dvars].notna().sum(axis=1)
# ipt['pt_i_total'] = np.where(ipt['pt_i_total'] == 0, non_nat_counts, ipt['pt_i_total'])

In [35]:
# ipt23 = filter_df_by_date(ipt, ipt_dataset.dvars, threshold)
# ipt23 = filter_df_by_date(ipt23, ipt_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# ipt_mask = (ipt23[ipt_dataset.dvars] > threshold) & (ipt23[ipt_dataset.dvars] < upper_threshold)
# ipt23_latestdate = ipt23[ipt_dataset.dvars].where(ipt_mask).max(axis=1)
# ipt23.insert(4, "service", "ipt")
# ipt23.insert(5, "date", ipt23_latestdate)
# ipt23 = ipt23[['rid', 'fcid', 'service', 'date', 'pt_i_total']]

In [36]:
# ipt23 = pd.merge(ipt23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ipt23 = pd.merge(ipt23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [37]:
# ipt23_path = os.path.join(root, data, processed, "unvfvt23", "iptBeneficiaries23.csv")
# ipt23.to_csv(ipt23_path, index=False)

## PHYSIOTHERAPY INTAKE ASSESSMENT PROCESSING

In [38]:
# ptint_dataset = Dataset(config_file, ptint_sheet)
# ptint = pd.read_csv(ptint_path)
# ptint = dtype_trans(ptint, ptint_dataset)

In [39]:
# ptint23 = filter_df_by_date(ptint, ptint_dataset.dvars, threshold)
# ptint23 = filter_df_by_date(ptint23, ptint_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# ptint23_latestdate = ptint23[['pt_nt_s1', 'pt_nt_s2']].max(axis=1)
# _ptint23 = ptint23.copy()
# _ptint23.insert(4, 'service', 'ptint')
# _ptint23.insert(5, 'date', ptint23_latestdate)
# _ptint23 = _ptint23[['rid', 'fcid', 'service', 'date']]
# _ptint23 = filter_df_by_date(_ptint23, ['date'], threshold)

In [40]:
# _ptint23 = pd.merge(_ptint23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# _ptint23 = pd.merge(_ptint23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [41]:
# ptint23_path = os.path.join(root, data, processed, "unvfvt23", "ptintBeneficiaries23.csv")
# _ptint23.to_csv(ptint23_path, index=False)

## PHYSIOTHERAPY RE-INTAKE ASSESSMENT PROCESSING

In [42]:
# ptreint23 = filter_df_by_date(ptint23, ['pt_nt_re'], threshold)
# ptreint23.insert(4, 'service', 'ptreint')
# ptreint23 = ptreint23.rename(columns={'pt_nt_re': 'date'})
# ptreint23 = ptreint23[['rid', 'fcid', 'service', 'date']]

In [43]:
# ptreint23_path = os.path.join(root, data, processed, "unvfvt23", "ptreintBeneficiaries23.csv")
# ptreint23.to_csv(ptreint23_path, index=False)

## PHYSIOTHERAPY FOLLOW-UP ASSESSMENT PROCESSING

In [44]:
# ptfua_dataset = Dataset(config_file, ptfua_sheet)
# ptfua = pd.read_csv(ptfua_path)
# ptfua = dtype_trans(ptfua, ptfua_dataset)

In [45]:
# ptfua323 = filter_df_by_date(ptfua, ['fu_3m'], threshold)
# ptfua323.insert(4, "service", "ptfua3")
# ptfua323 = ptfua323.rename(columns={'fu_3m': 'date'})
# ptfua323 = ptfua323[['rid', 'fcid', 'service', 'date']]
# ptfua323 = pd.merge(ptfua323, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ptfua323 = pd.merge(ptfua323, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ptfua323_path = os.path.join(root, data, processed, "unvfvt23", "ptfua3Beneficiaries23.csv")
# ptfua323.to_csv(ptfua323_path, index=False)

In [46]:
# ptfua623 = filter_df_by_date(ptfua, ['fu_6m'], threshold)
# ptfua623 = filter_df_by_date(ptfua623, ['fu_6m'], threshold=upper_threshold, comparison_type='b')
# ptfua623.insert(4, "service", "ptfua6")
# ptfua623 = ptfua623.rename(columns={'fu_6m': 'date'})
# ptfua623 = ptfua623[['rid', 'fcid', 'service', 'date']]
# ptfua623 = pd.merge(ptfua623, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ptfua623 = pd.merge(ptfua623, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ptfua623_path = os.path.join(root, data, processed, "unvfvt23", "ptfua6Beneficiaries23.csv")
# ptfua623.to_csv(ptfua623_path, index=False)

In [47]:
# ptfua1223 = filter_df_by_date(ptfua, ['fu_12m'], threshold)
# ptfua1223 = filter_df_by_date(ptfua1223, ['fu_12m'], threshold=upper_threshold, comparison_type='b')
# ptfua1223.insert(4, "service", "ptfua12")
# ptfua1223 = ptfua1223.rename(columns={'fu_12m': 'date'})
# ptfua1223 = ptfua1223[['rid', 'fcid', 'service', 'date']]
# ptfua1223 = pd.merge(ptfua1223, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ptfua1223 = pd.merge(ptfua1223, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ptfua1223_path = os.path.join(root, data, processed, "unvfvt23", "ptfua12Beneficiaries23.csv")
# ptfua1223.to_csv(ptfua1223_path, index=False)

# CONSOLIDATION

In [63]:
for file in os.listdir(unvfvt23):
    if file.endswith(".csv"):
        lst = []
        file_path = os.path.join(unvfvt23, file)
        df = pd.read_csv(file_path)
        for col in df.columns:
            lst.append(col)
        print(file, df.shape)
        print(lst)

gcBeneficiaries23.csv (81, 19)
['rid', 'fcid', 'service', 'date', 'gc_total', 'family_of_missing', 'survivor_of_detention', 'primary_torture', 'secondary_torture', 'sexual_violation_a', 'sexual_violation_p', 'hrd', 'journalist', 'wov', 'stgbv', 'lgbti', 'other', 'sex', 'age']
gptBeneficiaries23.csv (78, 19)
['rid', 'fcid', 'service', 'date', 'pt_g_total', 'family_of_missing', 'survivor_of_detention', 'primary_torture', 'secondary_torture', 'sexual_violation_a', 'sexual_violation_p', 'hrd', 'journalist', 'wov', 'stgbv', 'lgbti', 'other', 'sex', 'age']
intBeneficiaries23.csv (61, 18)
['rid', 'fcid', 'service', 'date', 'family_of_missing', 'survivor_of_detention', 'primary_torture', 'secondary_torture', 'sexual_violation_a', 'sexual_violation_p', 'hrd', 'journalist', 'wov', 'stgbv', 'lgbti', 'other', 'sex', 'age']
iptBeneficiaries23.csv (23, 19)
['rid', 'fcid', 'service', 'date', 'pt_i_total', 'family_of_missing', 'survivor_of_detention', 'primary_torture', 'secondary_torture', 'sexual_vi